In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
import numpy as np
print(tf.__version__)
print(tf.keras.__version__)

2.4.1
2.4.0


## 2、构建简单模型
### 2.1 模型堆叠
首先构建一个序列堆叠的网络模型，使用 `tf.keras.Sequential` 结构初始化 `model`， 并且通过 `model.add` 一层一层堆叠

In [2]:
model = tf.keras.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(72,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

### 2.2 layer 参数配置
了解一下 `tf.keras.layers`中一系列参数：
- `activation`
- `kernel_initializer` 和 `bias_initializer` ：
    - 创建层权重（核和偏差）的初始化方法。此参数是一个名称或可调用对象，默认为 "Glorot uniform" 初始化器。
- `kernel_regularizer` 和 `bias_regularizer`：
    - 应用层权重（核和偏差）的正则化方案，例如 L1 或 L2 正则化。默认情况下，系统不会应用正则化函数。


## 3、训练和评估
### 3.1 设置训练流程
现在有了堆叠好的模型，在正式开始训练模型之前，还需要使用 `model.compile` 配置一下训练流程：

In [3]:
model.compile(optimizer=keras.optimizers.Adam(lr=0.001),
             loss = keras.losses.categorical_crossentropy,
             metrics=[keras.metrics.categorical_crossentropy])

### 3.2 compile 参数配置
参数：
- `optimizer`:
    - 可以在调用 `model.compile()` 之前初始化一个优化器对象，然后传入该函数。也可以在调用 `model.compile()` 时传递一个预定义优化器名。所有优化器可以在括号中输入 `lr` 参数，也可以输入 `clipnorm` 和 `clipvalue` 进行梯度裁剪。预定义的优化器有如下几种：
    
        - SGD：keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
        - RMSprop：keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-06)
        - Adagrad：keras.optimizers.Adagrad(lr=0.01, epsilon=1e-06)
        - Adadelta：keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-06)
        - Adam：keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
        - Adamax：keras.optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
        - Nadam：keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
        
- `loss`
- `metrics`:
    - 与loss类似，不过此时的函数只用于模型性能评估但是不会用于训练，多和loss使用一样的函数。
        

### 3.3 随机生成数据进行训练

#### 3.3.1 使用 numpy 输入数据并训练

In [4]:
train_x = np.random.random((1000, 72))
train_y = np.random.random((1000, 10))

val_x = np.random.random((200, 72))
val_y = np.random.random((200, 10))

model.fit(train_x, train_y, epochs=10,batch_size=10, validation_data=(val_x, val_y))

Epoch 1/10
100/100 [==============================] - 1s 3ms/step - loss: 13.0822 - categorical_crossentropy: 13.0822 - val_loss: 23.7198 - val_categorical_crossentropy: 23.7198
Epoch 2/10
100/100 [==============================] - 0s 932us/step - loss: 28.4171 - categorical_crossentropy: 28.4171 - val_loss: 48.4061 - val_categorical_crossentropy: 48.4061
Epoch 3/10
100/100 [==============================] - 0s 903us/step - loss: 53.5981 - categorical_crossentropy: 53.5981 - val_loss: 61.7865 - val_categorical_crossentropy: 61.7865
Epoch 4/10
100/100 [==============================] - 0s 908us/step - loss: 45.9601 - categorical_crossentropy: 45.9601 - val_loss: 24.5644 - val_categorical_crossentropy: 24.5644
Epoch 5/10
100/100 [==============================] - 0s 900us/step - loss: 42.3531 - categorical_crossentropy: 42.3531 - val_loss: 104.8015 - val_categorical_crossentropy: 104.8015
Epoch 6/10
100/100 [==============================] - 0s 895us/step - loss: 92.9879 - categorical_cr

#### 3.3.2 使用 tf.data 输入数据并训练
除了使用 numpy 输入，也可以通过 `tf.data`将训练和验证集拼接够输入：

In [5]:
dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
dataset = dataset.batch(32).repeat()

val_dataset = tf.data.Dataset.from_tensor_slices((val_x, val_y))
val_dataset = val_dataset.batch(32).repeat()

model.fit(dataset, epochs=10, steps_per_epoch=30, 
          validation_data=val_dataset, validation_steps=3, shuffle=True)

Epoch 1/10
30/30 [==============================] - 0s 4ms/step - loss: 489.3187 - categorical_crossentropy: 489.3187 - val_loss: 781.5093 - val_categorical_crossentropy: 781.5093
Epoch 2/10
30/30 [==============================] - 0s 2ms/step - loss: 419.5004 - categorical_crossentropy: 419.5004 - val_loss: 379.0138 - val_categorical_crossentropy: 379.0138
Epoch 3/10
30/30 [==============================] - 0s 2ms/step - loss: 408.3172 - categorical_crossentropy: 408.3172 - val_loss: 596.7084 - val_categorical_crossentropy: 596.7084
Epoch 4/10
30/30 [==============================] - 0s 2ms/step - loss: 479.1322 - categorical_crossentropy: 479.1322 - val_loss: 422.1902 - val_categorical_crossentropy: 422.1902
Epoch 5/10
30/30 [==============================] - 0s 2ms/step - loss: 402.6422 - categorical_crossentropy: 402.6422 - val_loss: 893.9067 - val_categorical_crossentropy: 893.9067
Epoch 6/10
30/30 [==============================] - 0s 2ms/step - loss: 673.0088 - categorical_cross

#### 3.3.3  model.fit 参数配置
- x：输入数据。numpy array
- y：标签，numpy array
- batch_size
- epochs
- steps_per_epoch：整数，含义为每个 epoch 分割成多少个 batch_size
- validation_data：形式为 （x, y）的 tuple，是指定的验证集
- shuffle：是否打乱

### 3.4 评估与预测

In [6]:
test_x = np.random.random((1000, 72))
test_y = np.random.random((1000, 10))

model.evaluate(test_x, test_y, batch_size=32)

test_data = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_data = test_data.batch(32).repeat()

model.evaluate(test_data, steps=30)

result = model.predict(test_x, batch_size=32)

30/30 [==============================] - 0s 1ms/step - loss: 805.8557 - categorical_crossentropy: 805.8557


## 4、保存和恢复

### 4.1 权重的保存和读取
通过 `model.save_weights`和`model.load_weights`来进行权重的保存和读取，参数为文件路径

In [7]:
model.save_weights('./weights/only_weights')

In [8]:
model.load_weights('./weights/only_weights')

In [9]:
model.save_weights('./weights/only_weights.h5')

In [10]:
model.load_weights('./weights/only_weights.h5')

### 4.2 保存整个模型
可以通过 `model.save` 和 `tf.keras.model.load_model` 命令保存和读取整个模型：

In [11]:
model.save('./models/all_models.h5')

In [12]:
model = tf.keras.models.load_model('./models/all_models.h5')